![picture](https://prowly-uploads.s3.eu-west-1.amazonaws.com/uploads/4626/assets/71776/large_logo_wsb_poziom.jpg)

# WPROWADZENIE DO ALGORYTMÓW GŁĘBOKIEGO UCZENIA MASZYNOWEGO - DEEP LEARNING

## Wprowadzenie do sieci neuronowych w pythonie:
<ul>
    <li>Jednowarstwowa sieć neuronowa - step by step</li>
    <li>Dwuwarstwowe sieci neuronowe</li>
    <li>Wprowadzenie do sieci neuronowych z tensorflow Keras</li>
    <li>Aproksymacja funkcji przy użyciu sieci neuronowych</li>
    <li>Wpływ parametrów na efektywność sieci neuronowych<ul>
        <li>liczba warstw</li>
        <li>liczba neuronów w warstwie</li>
        <li>funkcja aktywacji</li>
        <li>krok uczenia</li>
    </ul>
    <li>Case study - problem regresji</li>
    <li>Case study - problem klasyfikacji</li>
    </li>
</ul>

# Jednowarstwowe sieci neuronowe + definicja zbiorów


# Biblioteki

In [ ]:
!pip install --upgrade statsmodels

In [ ]:
!pip install dalex

In [ ]:
import dalex

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# Zdefiniujmy podstawowe funcje aktywacji oraz ich pochodne
def sigmoid(x):
    return (1/(1+np.exp(-x)))
def relu(x):
    return np.maximum(0,x)
def pochodna_sigmoid(x):
    return (sigmoid(x)*(1-sigmoid(x)))
def pochodna_relu(x):
    return ((x>0)*1)
def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
def pochodna_tanh(x):
    t=(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    dt=1-t**2
    return dt

In [ ]:
# Zdefiniuj funkcje kosztu oraz ich pochodne
def funkcja_kosztu(x,y):
    return (pow((y-x),2)/2).sum()
def pochodna_funkcji_kosztu(x,y):
    return (x-y)

In [ ]:
rng = np.random.RandomState(1)
def sample_spherical(npoints, ndim=3,random_state=1,normalize=True,noise = True):
    np.random.seed(random_state)
    vec = np.random.randn(npoints,ndim)
    vec /= np.reshape(np.linalg.norm(vec, axis=1),[npoints,1])
    if noise :
      vec = vec + 0.1*rng.randn(npoints,ndim)
    if normalize :
      vec[:,0] = (vec[:,0]-min(vec[:,0]))/(max(vec[:,0]-min(vec[:,0])))
      vec[:,1] = (vec[:,1]-min(vec[:,1]))/(max(vec[:,1]-min(vec[:,1])))
    return vec
def sinusoida(npoints,random_state=1):
    np.random.seed(random_state)
    vec = np.zeros((npoints,2))
    vec[:,0] = np.arange(-2*np.pi,2*np.pi,4*np.pi/npoints)
    vec[:,1] = np.sin(vec[:,0])+0.1*rng.randn(npoints,1)[:,0]
    vec[:,0] = (vec[:,0]-min(vec[:,0]))/(max(vec[:,0]-min(vec[:,0])))
    vec[:,1] = (vec[:,1]-min(vec[:,1]))/(max(vec[:,1]-min(vec[:,1])))
    return(vec)
def skupiska(npoints,stand_devs,random_state=1):
    np.random.seed(random_state)
    skupisko_1 = stand_devs[0] * rng.randn(npoints, 2)+15
    skupisko_2 = stand_devs[1] * rng.randn(npoints, 2)-10
    skupisko_3 = stand_devs[2] * rng.randn(npoints, 2)+[30,-30]  
    trzy_skupiska = np.r_[skupisko_1,skupisko_2,skupisko_3]
    trzy_skupiska[:,0] = (trzy_skupiska[:,0]-min(trzy_skupiska[:,0]))/(max(trzy_skupiska[:,0]-min(trzy_skupiska[:,0])))
    trzy_skupiska[:,1] = (trzy_skupiska[:,1]-min(trzy_skupiska[:,1]))/(max(trzy_skupiska[:,1]-min(trzy_skupiska[:,1])))
    return trzy_skupiska
def add_noise(dataset, prop, low_bound, high_bound, min_dist = 0.5):
  rng = np.random.RandomState(1)
  noise = rng.uniform(low=low_bound,high=high_bound,size=dataset.shape)
  remove_from_noise =[]
  for i in range(len(noise)):
    distances = np.linalg.norm(dataset-noise[i],axis=1)
    closest_neig = distances.min()
    if (closest_neig<min_dist):
      remove_from_noise.append(i)
  noise = np.delete(noise, remove_from_noise,axis = 0)
  noise = noise[0:round(len(dataset)*prop),]
  dataset = np.r_[dataset,noise]
  return dataset

In [ ]:
# Zdefiniujmy proste zbiore danych, których użyjemy w procesie uczenia się sieci neuronowych
zbiór_liniowy = np.zeros([100,2])
zbiór_liniowy[:,0] = [x/100 for x in range(100)]
zbiór_liniowy[:,1] = [2*x+3 for x in zbiór_liniowy[:,0]]
plt.plot(zbiór_liniowy[:,0],zbiór_liniowy[:,1])

In [ ]:
# Zdefiniujmy proste zbiore danych, których użyjemy w procesie uczenia się sieci neuronowych
zbior_separowalny = np.zeros([100,2])
zbior_separowalny[:,0] = [x/100 for x in range(100)]
zbior_separowalny[:50,1] = np.random.rand(50)+5
zbior_separowalny[50:,1] = np.random.rand(50)-5
zbior_separowalny[:,0] = (zbior_separowalny[:,0]-min(zbior_separowalny[:,0]))/(max(zbior_separowalny[:,0]-min(zbior_separowalny[:,0])))
zbior_separowalny[:,1] = (zbior_separowalny[:,1]-min(zbior_separowalny[:,1]))/(max(zbior_separowalny[:,1]-min(zbior_separowalny[:,1])))
plt.scatter(zbior_separowalny[:,0],zbior_separowalny[:,1])

![picture](https://drive.google.com/uc?id=150xbihT0g4QkSHTaE4PR6U7tM9kglMfk)

In [ ]:
# Krok 1 - ustal początkowe wartości wag neuronów oraz przesunięć
def ustal_poczatkowe_wartosci_1_warstwa(wymiarowość_zbioru,liczba_neuronow):
    np.random.seed(1) # synchronizacja generatora liczb losowych u wszystkich użytkowników kodu
    w = np.random.rand(wymiarowość_zbioru,liczba_neuronow) # macierz wartości losowych wag 
    b = np.random.rand(liczba_neuronow) # wektor przesunięcia (bias)
    return w,b

In [ ]:
# Krok 2 - wylicz wartości f(wx+b)
def krok_wprzod(x,w,b,funkcja_aktywacji):
    z = np.dot(w,x)+b # iloczyn wag i obserwacji plus przesunięcie
    y_hat = funkcja_aktywacji(z) # wartość wyjściowa sieci - nałożenie funkcji aktywacji na neuron
    return z,y_hat

Wiemy, w jaki sposób wartości <i>w, b</i> oraz <i>X</i> wpływają na <i>y_hat</i>. W procesie propagacji wstecznej interesuje nas, jaki wpływ na wartość funkcji kosztu mają zmiany wartości <i>w</i> oraz <i>b</i>. 

Intuicyjnie: prowadzimy pewną firmę, naszym celem jest jej maksymalny rozwój. Chcemy, aby koszta całkowicie pokrywały budżet - nie zarabiamy, ale firma jest coraz większa. Niech X będzie pracownikami, W będzie wypłatą pracowników w firmie. Funkcja aktywacji oblicza ich koszty utrzymania (y_hat). Niech y będzie budżetem. Jako błąd oznaczamy różnicę między y i y_hat. Jeżeli różnica jest wysoka, budżet przewyższa koszta - nie wykorzystujemy potencjału w całości, więc należy zwiększyć wartość w, co zmotywuje pracowników. Zwiększamy do momentu, aż koszta nie przekroczą budżetu - jeżeli tak się stanie, zmniejszamy wypłatę pracowników. Szukamy optymalnego rozwiązania, czyli minimum funkcji kosztu. A co to stusnek zmiany wartości funkcji f(z) w zależności od zmiany w? Znana, i lubiana, pochodna.


![picture](https://drive.google.com/uc?id=1lr5NvMGI0uwMgP_cjOLv9iEE3JzsKFrN)

In [ ]:
# Krok 3 - aktualizacja wartości wag neuronów, czyli proces propagacji wstecznej
# bardzo polecam obejrzeć min. 5 razy film : https://www.youtube.com/watch?v=tIeHLnjs5U8
def roznica_wag_i_przesuniec(x,y,z,y_hat,pochodna_funkcja_kosztu,pochodna_funkcja_aktywacji):
    dw = x.T * (pochodna_funkcja_kosztu(y_hat, y) * pochodna_funkcja_aktywacji(z))
    db = np.sum(pochodna_funkcja_kosztu(y_hat, y) * pochodna_funkcja_aktywacji(z),axis = 0)
    return dw,db

Warto zwrócić uwagę na to, jakie wartości wpływają na wartość aktualizacji wag neuronów czy przesunięć. Oprócz różnicy między y_hat oraz y czy wektorem wejściowym x, używana jest wartość pochodnej funkcji aktywacji. 

In [ ]:
def aktualizacja_wag(w,b,dw,db,lr):
    w-=lr*dw # w = w-lr*w
    b-=lr*db
    return w,b

In [ ]:
### Trening sieci neuronowej jako klasy

class siec_neuronowa_1_warstwa():
  def __init__(
      self,
      liczba_neuronow: int,
      liczba_iteracji: int,
      funkcja_aktywacji: callable,
      pochodna_funkcji_aktywacji: callable,
      funkcja_kosztu: callable,
      pochodna_funkcji_kosztu: callable

  ):
    self.liczba_neuronow = liczba_neuronow
    self.liczba_iteracji = liczba_iteracji
    self.funkcja_aktywacji = funkcja_aktywacji
    self.pochodna_funkcji_aktywacji = pochodna_funkcji_aktywacji
    self.funkcja_kosztu = funkcja_kosztu
    self.pochodna_funkcji_kosztu = pochodna_funkcji_kosztu
    self.error = []
    self.curr_iter = 0
  
  def ustal_poczatkowe_wagi(self, n_dim : int):
    self.w,self.b = ustal_poczatkowe_wartosci_1_warstwa(n_dim, self.liczba_neuronow)
  def ustal_krok_uczenia(self, start,stop):
    self.lr = np.linspace(start, stop, self.liczba_iteracji)
  def wykonaj_krok_wprzod(self,x):
    self.z,self.y_hat = krok_wprzod(x,self.w,self.b, self.funkcja_aktywacji)
  def oblicz_blad(self,y):
    self.error.append(funkcja_kosztu(self.y_hat,y))
  def propagacja_wsteczna(self,x,y):
    dw,db = roznica_wag_i_przesuniec(x, y, self.z, self.y_hat, self.pochodna_funkcji_kosztu, self.pochodna_funkcji_aktywacji)
    self.w,self.b = aktualizacja_wag(self.w,self.b,dw,db,self.lr[self.curr_iter])
    self.curr_iter += 1
  def train_nn(self,data):
    fig = go.Figure()
    for i in range(self.liczba_iteracji):
      wylosuj_obs = np.int(np.floor(np.random.rand(1)*100)[0])
      x = data[wylosuj_obs,0]
      y = data[wylosuj_obs,1]
      self.wykonaj_krok_wprzod(x)
      self.oblicz_blad(y)
      self.propagacja_wsteczna(x,y)
      if i%1000 == 999:
        fig.add_trace(go.Scatter(x=data[:,0], y=self.funkcja_aktywacji(self.w*data[:,0]+self.b).reshape(100),
                    mode='lines',
                    name='Predykcja po '+str(i+1)+' iteracjach'))
    fig.add_trace(go.Scatter(x=data[:,0], y=data[:,1],
                    mode='markers',
                    name='dane'))
    fig.show()

## Przykład 1 - dopasowanie sieci do funkcji liniowej

In [ ]:
siec_zbior_liniowy = siec_neuronowa_1_warstwa(
    1,
    10000,
    funkcja_aktywacji = relu,
    pochodna_funkcji_aktywacji = pochodna_relu,
    funkcja_kosztu = funkcja_kosztu,
    pochodna_funkcji_kosztu = pochodna_funkcji_kosztu
    )

In [ ]:
siec_zbior_liniowy.ustal_poczatkowe_wagi(1)
siec_zbior_liniowy.ustal_krok_uczenia(0.0001,0.0001)
siec_zbior_liniowy.train_nn(zbiór_liniowy)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y = siec_zbior_liniowy.error,
        x = np.linspace(0,siec_zbior_liniowy.liczba_iteracji,siec_zbior_liniowy.liczba_iteracji)
        )
)
fig.show()

### Ćwiczenie
Sprawdź, w jaki sposób zmieni się błąd predykcji i dopasowanie sieci do danych gdy:<ul>
<li>zwiększysz początkowy learning rate,</li>
<li>zwiększysz liczbę iteracji.</li>

In [ ]:
# Zwiększona liczba iteracji


# Dwuwarstwowe sieci neuronowe + batch training

![picture](https://drive.google.com/uc?id=1kST4dC_-uZMMtenjIWSQzzBRxrgK0ZFN)

In [ ]:
# Krok 1 - ustal początkowe wartości wag neuronów oraz przesunięć
def ustal_poczatkowe_wartosci_wag(wymiarowość_zbioru, liczba_neuronow):
    np.random.seed(1)
    w = np.random.rand(wymiarowość_zbioru, liczba_neuronow)
    b = np.random.rand(liczba_neuronow)
    return w, b

In [ ]:
# Krok 2 - wylicz wartości f(wx+b)
def krok_wprzod_batch(x, w, b, funkcja_aktywacji):
    z = np.dot(x, w) + b
    y_hat = funkcja_aktywacji(z)
    return z, y_hat

In [ ]:
# Krok 3 - aktualizacja wartości wag neuronów, czyli proces propagacji wstecznej
def roznica_wag_i_przesuniec_warstwa_2(y, a_1, z_2, y_hat, pochodna_funkcja_kosztu,
                                   pochodna_funkcja_aktywacji_w2):
    # wektorem wejściowym w przypadku warstwy 2 jest wartość funkcji aktywacji dla neuronów warstwy pierwszej,
    # stąd zamiast x.T -> a_1.T
    dw = np.dot(a_1.T, (pochodna_funkcja_kosztu(y_hat, y) *
                        pochodna_funkcja_aktywacji_w2(z_2)))
    db = np.sum(pochodna_funkcja_kosztu(y_hat, y) *
                pochodna_funkcja_aktywacji_w2(z_2),
                axis=0)
    return dw, db

In [ ]:
# Krok 3 - aktualizacja wartości wag neuronów, czyli proces propagacji wstecznej
def roznica_wag_i_przesuniec_warstwa_1(x, y, w_2, z_2, z_1, y_hat, pochodna_funkcja_kosztu,
                                   pochodna_funkcja_aktywacji_w1,
                                   pochodna_funkcja_aktywacji_w2):
    # wzór na wartość aktualizacji wektora wag warstwy pierwszej w sieci neuronowej jest trochę bardziej skomplikowany
    # wynika on z tzw. chain rule
    # ponownie, odsyłam do linku https://www.youtube.com/watch?v=tIeHLnjs5U8
    dw = np.dot(x.T, (np.dot(
        (pochodna_funkcja_kosztu(y_hat, y) * pochodna_funkcja_aktywacji_w2(z_2)),
        w_2.T) * pochodna_funkcja_aktywacji_w1(z_1)))
    db = np.sum(np.dot(
        (pochodna_funkcja_kosztu(y_hat, y) * pochodna_funkcja_aktywacji_w2(z_2)),
        w_2.T) * pochodna_funkcja_aktywacji_w1(z_1),
                axis=0)
    return dw, db

In [ ]:
def aktualizacja_wag(w,b,dw,db,lr):
    w-=lr*dw
    b-=lr*db
    return w,b

In [ ]:
class siec_neuronowa_2_warstwy():
  def __init__(
      self,
      liczba_neuronow_w1: int,
      liczba_neuronow_w2: int,
      liczba_iteracji: int,
      funkcja_aktywacji_w1: callable,
      funkcja_aktywacji_w2: callable,
      pochodna_funkcji_aktywacji_w1: callable,
      pochodna_funkcji_aktywacji_w2: callable,
      funkcja_kosztu: callable,
      pochodna_funkcji_kosztu: callable

  ):
    self.liczba_neuronow_w1 = liczba_neuronow_w1
    self.liczba_neuronow_w2 = liczba_neuronow_w2
    self.liczba_iteracji = liczba_iteracji
    self.funkcja_aktywacji_w1 = funkcja_aktywacji_w1
    self.funkcja_aktywacji_w2 = funkcja_aktywacji_w2
    self.pochodna_funkcji_aktywacji_w1 = pochodna_funkcji_aktywacji_w1
    self.pochodna_funkcji_aktywacji_w2 = pochodna_funkcji_aktywacji_w2
    self.funkcja_kosztu = funkcja_kosztu
    self.pochodna_funkcji_kosztu = pochodna_funkcji_kosztu
    self.error = []
    self.curr_iter = 0
  
  def ustal_poczatkowe_wagi(self, n_dim : int):
    self.w_1,self.b_1 = ustal_poczatkowe_wartosci_wag(n_dim, self.liczba_neuronow_w1)
    self.w_2,self.b_2 = ustal_poczatkowe_wartosci_wag(self.liczba_neuronow_w1,self.liczba_neuronow_w2)
  def ustal_krok_uczenia(self, start,stop):
    self.lr = np.linspace(start, stop, self.liczba_iteracji)
  def wykonaj_krok_wprzod(self,x):
    self.z_1, self.a_1 = krok_wprzod_batch(x,self.w_1,self.b_1, self.funkcja_aktywacji_w1)
    self.z_2, self.y_hat = krok_wprzod_batch(self.a_1, self.w_2, self.b_2, self.funkcja_aktywacji_w2)
  def oblicz_blad(self,y):
    self.error.append(funkcja_kosztu(self.y_hat,y))
  def propagacja_wsteczna(self,x,y):
    dw_2,db_2 = roznica_wag_i_przesuniec_warstwa_2(
        y,
        self.a_1,
        self.z_2,
        self.y_hat,
        self.pochodna_funkcji_kosztu,
        self.pochodna_funkcji_aktywacji_w2)
    dw_1,db_1 = roznica_wag_i_przesuniec_warstwa_1(
        x, 
        y,
        self.w_2, 
        self.z_2, 
        self.z_1, 
        self.y_hat, 
        self.pochodna_funkcji_kosztu,
        self.pochodna_funkcji_aktywacji_w1,
        self.pochodna_funkcji_aktywacji_w2)
    self.w_2,self.b_2 = aktualizacja_wag(self.w_2,self.b_2,dw_2,db_2,self.lr[self.curr_iter])
    self.w_1,self.b_1 = aktualizacja_wag(self.w_1,self.b_1,dw_1,db_1,self.lr[self.curr_iter])
    self.curr_iter += 1
  def train_nn(self,data, plot_data = True):
    fig = go.Figure()
    for i in range(self.liczba_iteracji):
      x = data[:,:data.shape[1]-1].reshape([len(data),data.shape[1]-1])
      y = data[:,data.shape[1]-1].reshape([len(data),1])
      self.wykonaj_krok_wprzod(x)
      self.oblicz_blad(y)
      self.propagacja_wsteczna(x,y)
      if i%100 == 99:
        fig.add_trace(go.Scatter(x=data[:,0], y=self.y_hat.reshape(len(data)),
                    mode='lines',
                    name='Predykcja po '+str(i+1)+' iteracjach'))
    fig.add_trace(go.Scatter(x=data[:,0], y=data[:,1],
                    mode='markers',
                    name='dane'))
    if plot_data:
      fig.show()

## Przykład 2 - dopasowanie sieci do zbioru liniowo separowalnego

In [ ]:
siec_zbior_separowalny_w2 = siec_neuronowa_2_warstwy(
    liczba_neuronow_w1 = 5,
    liczba_neuronow_w2 = 1,
    liczba_iteracji = 1500,
    funkcja_aktywacji_w1 = sigmoid,
    funkcja_aktywacji_w2 = relu,
    pochodna_funkcji_aktywacji_w1 = pochodna_sigmoid,
    pochodna_funkcji_aktywacji_w2 = pochodna_relu,
    funkcja_kosztu = funkcja_kosztu,
    pochodna_funkcji_kosztu = pochodna_funkcji_kosztu
    )

In [ ]:
siec_zbior_separowalny_w2.ustal_poczatkowe_wagi(1)
siec_zbior_separowalny_w2.ustal_krok_uczenia(0.001, 0.0001)
siec_zbior_separowalny_w2.train_nn(zbior_separowalny)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y = siec_zbior_separowalny_w2.error,
        x = np.linspace(0,siec_zbior_separowalny_w2.liczba_iteracji,siec_zbior_separowalny_w2.liczba_iteracji)
        )
)
fig.show()

### Zadanie 1 <br>
Wprowadź następujące zmiany w kodzie : <ul>
<li>funkcje aktywacji - 1.relu 2.sigmoid, lr (0.1,0.001), (0.01,0.001), liczba neuronów 2, 5,10 </li>
<li>obie funkcje aktywacji relu, liczba neuronów 2,5,10, lr - ?</li>
<li>funkcje aktywacji - 1.sigmoid 2.relu , lr i liczba neuronów ?</li>
</ul>

W jaki sposób zmiany wpływają na tempo uczenia sieci? 
<br><br>
*Co jest powodem takiego zachowania?

In [ ]:
# 1a)
siec_zbior_separowalny_w2 = siec_neuronowa_2_warstwy(
    liczba_neuronow_w1 = 5,
    liczba_neuronow_w2 = 1,
    liczba_iteracji = 25000,
    funkcja_aktywacji_w1 = sigmoid,
    funkcja_aktywacji_w2 = sigmoid,
    pochodna_funkcji_aktywacji_w1 = pochodna_sigmoid,
    pochodna_funkcji_aktywacji_w2 = pochodna_sigmoid,
    funkcja_kosztu = funkcja_kosztu,
    pochodna_funkcji_kosztu = pochodna_funkcji_kosztu
    )
siec_zbior_separowalny_w2.ustal_poczatkowe_wagi(1)
siec_zbior_separowalny_w2.ustal_krok_uczenia(0.5, 0.001)
siec_zbior_separowalny_w2.train_nn(zbior_separowalny)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y = siec_zbior_separowalny_w2.error,
        x = np.linspace(0,siec_zbior_separowalny_w2.liczba_iteracji,siec_zbior_separowalny_w2.liczba_iteracji)
        )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y = siec_zbior_separowalny_w2.error,
        x = np.linspace(0,siec_zbior_separowalny_w2.liczba_iteracji,siec_zbior_separowalny_w2.liczba_iteracji)
        )
)
fig.show()

In [ ]:
# 1b)

In [ ]:
# 1c)

## Przykład 3 - XOR

In [ ]:
XOR = np.zeros([400,4])

In [ ]:
XOR[:100,1:3] = np.array([np.random.rand(100)/5,
                           np.random.rand(100)/5]).reshape(100,2)
XOR[:100,2] = XOR[:100,2]+1                      
XOR[100:200,1:] = np.array([1+np.random.rand(100)/3,
                            1+np.random.rand(100)/2,
                            1+np.random.rand(100)/10]).reshape(100,3)
XOR[200:300,[0,2,3]] = np.array([1+np.random.rand(100)/10,
                            1+np.random.rand(100)/2,
                            1+np.random.rand(100)/3]).reshape(100,3)
XOR[300:,:2] = np.array([1+np.random.rand(100)/3,
                            1+np.random.rand(100)/5]).reshape(100,2)

In [ ]:
x = XOR[:,:3]
y = XOR[:,3].reshape([400,1])

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x[:,0], y=x[:,1], z=x[:,2],
                                   mode='markers',marker = dict(
                                   color = np.concatenate(y),
                                   colorbar=dict(
                                       title = 'y'
                                   ),
                                    colorscale="Viridis"))])
fig.show()

In [ ]:
siec_XOR = siec_neuronowa_2_warstwy(
    liczba_neuronow_w1 = 1,
    liczba_neuronow_w2 = 1,
    liczba_iteracji = 1500,
    funkcja_aktywacji_w1 = relu,
    funkcja_aktywacji_w2 = sigmoid,
    pochodna_funkcji_aktywacji_w1 = pochodna_relu,
    pochodna_funkcji_aktywacji_w2 = pochodna_sigmoid,
    funkcja_kosztu = funkcja_kosztu,
    pochodna_funkcji_kosztu = pochodna_funkcji_kosztu
    )

In [ ]:
siec_XOR.ustal_poczatkowe_wagi(x.shape[1])
siec_XOR.ustal_krok_uczenia(0.001, 0.0001)
siec_XOR.train_nn(XOR, plot_data = False)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x[:,0], y=x[:,1], z=x[:,2],
                                   mode='markers',marker = dict(
                                   color = np.concatenate(siec_XOR.y_hat),
                                   colorbar=dict(
                                       title = 'y_hat'
                                   ),
                                    colorscale="Viridis")
                                   )])
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        y = siec_XOR.error,
        x = np.linspace(0,siec_XOR.liczba_iteracji,siec_XOR.liczba_iteracji)
        )
)
fig.show()

###Zadanie 2 <br>
Przetestuj kombinacje:<ul>
<li>5 neuronów w warstwie, obie funkcje aktywacji relu, lr = 0.1</li>
<li>5 neuronów w warstwie, obie funkcje aktywacji relu, lr = 0.01</li>
<li>5 neuronów w warstwie, obie funkcje aktywacji sigmoid, lr = 0.1</li>
<li>5 neuronów w warstwie, obie funkcje aktywacji sigmoid, lr = 0.01</li>
</ul>
Wytłumacz różnice między wariantami.



In [ ]:
# 2a)

In [ ]:
# 2b)

In [ ]:
# 2c)

In [ ]:
# 2d)

# Wprowadzenie do sieci neuronowych w tensorflow Keras

In [ ]:
# Ogólny schemat podstawowych modeli sieci neuronowych w Keras :

model = Sequential()  # Inicjacja sieci neuronowych
model.add(
    Dense(
        'liczba neuronow : int',  #dodajemy input layer (tutaj nazywamy to Dense)
        input_dim='liczba wymiarow w zbiorze wejsciowym : int',
        activation='funkcja aktywacji: function'
    )  # lista funkcji znajduje się na https://www.tensorflow.org/api_docs/python/tf/keras/activations
)  # aby użyć funkcji aktywacji, należy ją importować z tensorflow.keras.activations

model.add(
    Dense(
        'liczba neuronow : int',  # kolejne warstwy dodajemy w analogiczny sposób do wejściowej, z wyłączeniem argumentu o wymiarowości zbioru
        activation='funkcja aktywacji: function'))

# .....  dodajemy tyle ukrytych warstw, ile jest koniecze

model.add(
    Dense(
        'liczba neuronow : int',  # w warstwie wyjściowej liczba neuronów decyduje o wymiarowości predykcji 
        activation='funkcja aktywacji: function'))

# na etapie kompilowania określamy funkcję kosztu, metodę optymalizacji wag czy też metrykę,
# która będzie zapisywana podczas treningu
model.compile(
    loss='funkcja kosztu: function',
    optimizer=SGD(
        lr='współczynnik uczenia się: float'
    ),  # są inne opcje optymalizacji (np.Adam) - dla chętnych https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
    metrics=['mse']
)  # lista metryk https://www.tensorflow.org/api_docs/python/tf/keras/metrics

# Trening sieci neuronowych
history = model.fit(
    x='zbior wejsciowy: np.Array/pd.DataFrame/tf.data',  # zbiór wejściowy
    y='szukana y: np.array/pd.DataFrame/tf.data',  # oczekiwana wartość
    epochs='liczba iteracji: int',  # liczba iteracji
    batch_size='rozmiar wsadu: int',  # rozmiar próby w uczeniu wsadowym
    validation_split=
    'udział części walidacyjnej: float',  # podział zbioru na część treningową i walidacyjną, keras robi to za nas
    verbose='{0,1,2}: int'
)  # odpowiada za wiadomości zwrotne podczas trenignu modelu
# doczytaj inne argumenty .fit - https://www.tensorflow.org/api_docs/python/tf/keras/Sequential

predictions = model.predict(
    x='dane, dla ktorych chcemy wyliczyc predykcje: np.Array/pd.DataFrame/tf.data'
)

In [ ]:
from IPython.display import Image
Image(url='https://machinelearningknowledge.ai/wp-content/uploads/2020/12/Keras_Optimizers-min.gif?fbclid=IwAR0ErU_uLQcqR9niO0WnYrJ_P87ACslPfkl7iolAc8v1Nc9BV6PBA2d0ygM')

In [ ]:
Image(url='https://github.com/Jaewan-Yun/optimizer-visualization/blob/master/figures/movie11.gif?raw=true')

In [ ]:
Image(url='https://github.com/Jaewan-Yun/optimizer-visualization/blob/master/figures/movie9.gif?raw=true')

https://bl.ocks.org/EmilienDupont/raw/aaf429be5705b219aaaf8d691e27ca87/ <br>
https://github.com/Jaewan-Yun/optimizer-visualization

Do poczytania 

<b> https://www.tensorflow.org/api_docs/python/tf/keras - dokumentacja<br>
https://towardsdatascience.com/learning-process-of-a-deep-neural-network-5a9768d7a651 - podsumowanie sieci neuronowych <br>
https://playground.tensorflow.org/ - jak działają sieci w praktyce, świetne wizualizacje <br>
https://www.deeplearningbook.org/ - książka o deep learningu, pierwszy autor jest dyrektorem machine learningu w Apple, całość w html za darmo


# Biblioteki

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras import backend as K, metrics, layers, losses, optimizers

from tensorflow.keras.activations import elu, exponential, hard_sigmoid, linear, relu, sigmoid, softmax, tanh
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy, logcosh, mean_squared_error, poisson, mean_absolute_error
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adadelta, Adam, Nadam, RMSprop, SGD, Ftrl

## Przykład 5 - dopasowanie sieci do sinusoidy

In [ ]:
sinus = sinusoida(1000,1)

In [ ]:
plt.scatter(sinus[:, 0], sinus[:, 1], c='white',
                 s=20, edgecolor='k')
plt.show()

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=sigmoid))

model.add(Dense(64,
                activation=sigmoid))
model.add(Dense(32,
                activation=sigmoid))
model.add(Dense(16,
                activation=sigmoid))
model.add(Dense(8,
                activation=sigmoid))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(),
              metrics=['mse'])

history = model.fit(sinus[:, 0],
                    sinus[:, 1],
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=1)

predictions = model.predict(sinus[:, 0])

In [ ]:
plt.scatter(sinus[:, 0], predictions.reshape(1000), c='white',
                 s=20, edgecolor='k')
plt.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=relu))

model.add(Dense(64,
                activation=relu))
model.add(Dense(32,
                activation=relu))
model.add(Dense(16,
                activation=relu))
model.add(Dense(8,
                activation=relu))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(),
              metrics=['mse'])

history = model.fit(sinus[:, 0],
                    sinus[:, 1],
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(sinus[:, 0])

In [ ]:
plt.scatter(sinus[:, 0], predictions.reshape(1000), c='white',
                 s=20, edgecolor='k')
plt.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=linear))

model.add(Dense(64,
                activation=linear))
model.add(Dense(32,
                activation=linear))
model.add(Dense(16,
                activation=linear))
model.add(Dense(8,
                activation=linear))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(),
              metrics=['mse'])

history = model.fit(sinus[:, 0],
                    sinus[:, 1],
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(sinus[:, 0])

In [ ]:
plt.scatter(sinus[:, 0], predictions.reshape(1000), c='white',
                 s=20, edgecolor='k')
plt.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=tanh))

model.add(Dense(64,
                activation=tanh))
model.add(Dense(32,
                activation=tanh))
model.add(Dense(16,
                activation=tanh))
model.add(Dense(8,
                activation=tanh))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(),
              metrics=['mse'])

history = model.fit(sinus[:, 0],
                    sinus[:, 1],
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(sinus[:, 0])

In [ ]:
plt.scatter(sinus[:, 0], predictions.reshape(1000), c='white',
                 s=20, edgecolor='k')
plt.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=relu))

model.add(Dense(64,
                activation=relu))
model.add(Dense(32,
                activation=relu))
model.add(Dense(16,
                activation=relu))
model.add(Dense(8,
                activation=relu))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(
                  learning_rate = 0.5
                  ),
              metrics=['mse'])

history = model.fit(sinus[:, 0],
                    sinus[:, 1],
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(sinus[:, 0])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sinus[:, 0], y=predictions.reshape(1000),
            mode='markers',
            name='Predykcje'))
fig.add_trace(go.Scatter(x=sinus[:, 0], y=sinus[:, 1],
                    mode='markers',
                    name='Wartości rzeczywiste'))
fig.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=relu))

model.add(Dense(64,
                activation=relu))
model.add(Dense(32,
                activation=elu))
model.add(Dense(16,
                activation=elu))
model.add(Dense(8,
                activation=elu))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(
                  learning_rate = 0.9
              ),
              metrics=['mse'])

history = model.fit(sinus[:, 0],
                    sinus[:, 1],
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=1)

predictions = model.predict(sinus[:, 0])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sinus[:, 0], y=predictions.reshape(1000),
            mode='markers',
            name='Predykcje'))
fig.add_trace(go.Scatter(x=sinus[:, 0], y=sinus[:, 1],
                    mode='markers',
                    name='Wartości rzeczywiste'))
fig.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

Pytanie - dlaczego nie widać wartości historii uczenia się sieci? 

### Zadanie 3<br>
Oblicz predykcje dla wartości w przedziale [-5,5] (np.linspace(-5,5,300)) przy użyciu najlepszego modelu. 
Zwizualizuj predykcje przy użyciu plt.plot.

In [ ]:
###

In [ ]:
predictions = model.predict(np.linspace(-5,5,300))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.linspace(-5,5,300), y=predictions.reshape(300),
            mode='markers',
            name='Predykcje'))
fig.add_trace(go.Scatter(x=sinus[:, 0], y=sinus[:, 1],
                    mode='markers',
                    name='Wartości rzeczywiste'))
fig.show()

### Zadanie 4

Na podstawie przykładu 5 wytrenuj sieć, która w 1000 iteracji dopasuje się do badanego zbioru

In [ ]:
chmura_punktow_2d = skupiska(100, [1, 2, 1])

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=chmura_punktow_2d[:, 0],
               y=chmura_punktow_2d[:, 1],
               mode='markers'))
fig.show()

In [ ]:
x = chmura_punktow_2d[:,0].reshape([300,1])
y = chmura_punktow_2d[:,1].reshape([300,1])

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=1,
          activation=relu))

model.add(Dense(64,
                activation=relu))
model.add(Dense(32,
                activation=relu))
model.add(Dense(16,
                activation=relu))
model.add(Dense(8,
                activation=relu))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(
                  learning_rate = 0.2
                  ),
              metrics=['mse'])

history = model.fit(x,
                    y,
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(x)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=chmura_punktow_2d[:, 0],
               y=chmura_punktow_2d[:, 1],
               mode='markers',
               name = 'wartosci rzeczywiste'))
fig.add_trace(
    go.Scatter(x=chmura_punktow_2d[:, 0],
               y=predictions.reshape(300),
               mode='markers',
               name = 'predictions'))
fig.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

Oblicz predykcje modelu dla wartości np.linspace(0,1,1000) i zwizualizuj przy użyciu plotly

In [ ]:
predictions = model.predict(np.linspace(0,1,1000))

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=chmura_punktow_2d[:, 0],
               y=chmura_punktow_2d[:, 1],
               mode='markers',
               name = 'wartosci rzeczywiste'))
fig.add_trace(
    go.Scatter(x=np.linspace(0,1,1000),
               y=predictions.reshape(1000),
               mode='markers',
               name = 'predictions'))
fig.show()

# Prognozowanie

Wiemy już, że predykcje dla wartości spoza zakresu uczenia daleko odbiegały od kształtu. Sprawdźmy, czy sieć jest w stanie nauczyć się sezonowości sinusoidy do prognozowania. W tym celu:<ul>
<li>stworzymy nową sinusoidę, tym razem na większym zakresie x</li>
<li>przystosujemy zbiór do prognozowania na podstawie 50 ostatnich pomiarów</li>
<li>wytrenujemy model sieci neuronowej</li>
<li>sprawdzimy predykcje na przedziale spoza zakresu uczenia</li>

In [ ]:
rng = np.random.RandomState(1)

In [ ]:
def ext_sinusoida(npoints,random_state=1):
    np.random.seed(random_state)
    vec = np.zeros((npoints,2))
    vec[:,0] = np.arange(-5*np.pi,5*np.pi,10*np.pi/npoints)
    vec[:,1] = np.sin(vec[:,0])+0.1*rng.randn(npoints,1)[:,0]
    return(vec)

In [ ]:
sinus

In [ ]:
sinus = ext_sinusoida(1000)
sample_length = 50


X = sinus[:,1]
Y = sinus[sample_length:,1]

input_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
  X, None, sequence_length=sample_length, sequence_stride=1, batch_size = len(Y))
target_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
  Y, None, sequence_length=1, sequence_stride=1, batch_size = len(Y))

for batch in zip(input_dataset, target_dataset):
  inputs, targets = batch

In [ ]:
plt.scatter(sinus[:, 0], sinus[:, 1], c='white',
                 s=20, edgecolor='k')
plt.show()

In [ ]:
inputs

In [ ]:
sinus_df = pd.DataFrame(inputs.numpy())

In [ ]:
sinus_df['target'] = targets.numpy()

In [ ]:
sinus_df

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.8)


In [ ]:
model_forecast = Sequential()
model_forecast.add(
    tf.keras.layers.Flatten(),)
model_forecast.add(Dense(200,
                activation=relu))
model_forecast.add(Dense(100,
                activation=relu))
model_forecast.add(Dense(50,
                activation=relu))
model_forecast.add(Dense(35,
                activation=elu))
model_forecast.add(Dense(25,
                activation=elu))
model_forecast.add(Dense(10,
                activation=elu))
model_forecast.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model_forecast.compile(loss=mean_squared_error,
              optimizer=Adam(
                   learning_rate = lr_schedule
                  ),
              metrics=['mse'])

history = model_forecast.fit(sinus_df.drop('target',axis=1),
                    sinus_df.target,
                    epochs=300,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=1)

In [ ]:
predictions = model_forecast.predict(inputs.numpy())
fig = go.Figure()
fig.add_trace(go.Scatter(x=sinus[sample_length:, 0], y=predictions.reshape(len(Y)),
            mode='markers',
            name='Predykcje'))
fig.add_trace(go.Scatter(x=sinus[sample_length:, 0], y=sinus[sample_length:, 1],
                    mode='markers',
                    name='Wartości rzeczywiste'))
fig.show()

Sieć nauczyła się badanego zbioru. Czy jest w stanie przewidzieć wartości spoza zakresu uczenia?

In [ ]:
zakres = np.arange(4*np.pi,6*np.pi,2*np.pi/200)

X = np.sin(zakres)
input_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
  X, None, sequence_length=sample_length, sequence_stride=1, batch_size = 951)

for batch in input_dataset:
  inputs = batch

In [ ]:
inputs.numpy()

In [ ]:
predictions = model_forecast.predict(inputs.numpy())

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=zakres[50:], y=predictions.reshape(151),
            mode='markers',
            name='Predykcje'))
fig.add_trace(go.Scatter(x=zakres, y=np.sin(zakres),
            mode='markers',
            name='Wartości rzeczywiste'))
fig.show()

## Przykład 6 - predykcje dla okręgu, proste zależności 

In [ ]:
rng = np.random.RandomState(1)
sfera = sample_spherical(1000,2)
sfera_y = sfera[:,0] - sfera[:,1]
sfera_y = (sfera_y - min(sfera_y))/(max(sfera_y) - min(sfera_y))

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=sfera[:, 0], y=sfera[:, 1], z=sfera_y,
                                   mode='markers',marker = dict(
                                   color = sfera_y))])
fig.show()

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=2,
          activation=relu))
model.add(Dense(32,
                activation=elu))
model.add(Dense(8,
                activation=tanh))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(
                  learning_rate=0.01
                  ),
              metrics=['mse'])

history = model.fit(sfera,
                    sfera_y,
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(sfera)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=sfera[:, 0], y=sfera[:, 1], z=predictions.reshape(1000),
                                   mode='markers',marker = dict(
                                   color = predictions.reshape(1000)))])
fig.show()

In [ ]:
plt.plot(range(500),(history.history)['loss'])

## Przykład 7 - dokładność sieci dla skomplikowanych zależności między zmiennymi, zbiory o charakterze grupowym

In [ ]:
rng = np.random.RandomState(1)
trzy_skupiska = skupiska(1000, [3, 5, 7])
trzy_skupiska_y = np.r_[np.sum(5*(trzy_skupiska[:1000,:] - [0.5,0.87])**2,axis = 1),
                        np.sum(-1*(trzy_skupiska[1000:2000,:] - [0.2,0.55])**2,axis = 1),
                        np.sum((trzy_skupiska[2000:3000,:] - [0.7,0.25])**2,axis = 1)]
trzy_skupiska_y = (trzy_skupiska_y - min(trzy_skupiska_y))/(max(trzy_skupiska_y) - min(trzy_skupiska_y))

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=trzy_skupiska[:, 0], y=trzy_skupiska[:, 1], z=trzy_skupiska_y,
                                   mode='markers',marker = dict(
                                   color = trzy_skupiska_y))])
fig.show()

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=2,
          activation=relu))
model.add(Dense(48,
                activation=relu))
model.add(Dense(12,
                activation=relu))
 
model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))
 
model.compile(loss=mean_squared_error,
              optimizer=SGD(
                  learning_rate = 0.01
              ),
              metrics=['mse'])
 
history = model.fit(trzy_skupiska,
                    trzy_skupiska_y,
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)
 
predictions = model.predict(trzy_skupiska)

In [ ]:
plt.plot(range(500),(history.history)['loss'])

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=trzy_skupiska[:, 0], y=trzy_skupiska[:, 1], z=predictions.reshape(3000),
                                   mode='markers',marker = dict(
                                   color = predictions.reshape(3000)))])
fig.show()

### Zadanie 5 <br>
a) Zmień wartości learning rate na wyższe.<br>
b) Wytrenuj od początku powyższy model, powiększając dwukrotnie liczbę neuronów osobno w każdej warstwie (oprócz wyjściowej). W jakim przypadku osiągnięta została najniższa wartość mse, a w jakim najwyższa?<br>

In [ ]:
# 5a)

Spróbuj użyć jako optimizer poniższą wartość:

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.005,
    decay_steps=1000,
    decay_rate=0.8)

optimizer = Adam(
    learning_rate = lr_schedule
)

## Przykład 8 - tendencje do uogólniania zależności

In [ ]:
zbior_v = np.zeros([1000,2])
zbior_v[:,0] = [x/1000 for x in range(1000)]
rng = np.random.RandomState(1)
zbior_v[:500,1] = [-6*x+3 for x in zbior_v[:500,0]]+rng.randn(500)*rng.uniform(-1,1,500)
rng = np.random.RandomState(2)
zbior_v[500:,1] = [6*x-3 for x in zbior_v[500:,0]]+rng.randn(500)*rng.uniform(-1,1,500)
zbior_v_y = (zbior_v[:,0]-zbior_v[:,1])+rng.randn(1000)*rng.uniform(-1,1,1000)
zbior_v_y = (zbior_v_y - min(zbior_v_y))/(max(zbior_v_y) - min(zbior_v_y))
fig = go.Figure(data=[go.Scatter3d(x=zbior_v[:, 0], y=zbior_v[:, 1], z=zbior_v_y,
                                   mode='markers',marker = dict(
                                   color = zbior_v_y))])
fig.show()

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=2,
          activation=relu))
model.add(Dense(32,
                activation=elu))
model.add(Dense(8,
                activation=elu))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=SGD(),
              metrics=['mse'])

history = model.fit(zbior_v,
                    zbior_v_y,
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(zbior_v)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=zbior_v[:, 0], y=zbior_v[:, 1], z=predictions.reshape(1000),
                                   mode='markers',marker = dict(
                                   color = predictions.reshape(1000)))])
fig.show()

## Przykład 9 - tendencje do wyszukiwania ciągłych zależności

In [ ]:
rng = np.random.RandomState(1)
sinus = sinusoida(1000)
sinus_y = np.r_[
    [-2 * (x + y) -2 for x, y in zip(sinus[:200, 0], sinus[:200, 1])],
    [-6 * (1-x) + y*3 + 1 for x, y in zip(sinus[200:500, 0], sinus[200:500, 1])],
    [(1-x)*y - 6 * (x-y) - 3 for x, y in zip(sinus[500:, 0], sinus[500:, 1])]]
sinus_y = (sinus_y - min(sinus_y)) / (max(sinus_y) - min(sinus_y))

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=sinus[:, 0], y=sinus[:, 1], z=sinus_y,
                                   mode='markers',marker = dict(
                                   color = sinus_y))])
fig.show()

In [ ]:
model = Sequential()
model.add(Dense(128,
          input_dim=2,
          activation=relu))

model.add(Dense(64,
                activation=relu))
model.add(Dense(8,
                activation=relu))


model.add(Dense(1, activation=linear,
                kernel_initializer='normal'))

model.compile(loss=mean_squared_error,
              optimizer=Adam(
                  lr = 0.001
              ),
              metrics=['mse'])

history = model.fit(sinus,
                    sinus_y,
                    epochs=500,
                    batch_size=100,
                    validation_split=0.0,
                    verbose=0)

predictions = model.predict(sinus)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=sinus[:, 0], y=sinus[:, 1], z=predictions.reshape(1000),
                                   mode='markers',marker = dict(
                                   color = predictions.reshape(1000)))])
fig.show()

# Case Study - predykcja opłat za ubezpieczenie

#### Wgrywanie danych, EDA

Na zajęciach przeanalizujemy zbiór danych dostępny na <a href="https://www.kaggle.com/priyang/health-insurance-cost-prediction-using-ml">kaggle</a>. Obserwacje dotyczą wybranych opłat za ubezpieczenie zdrowotne na rynku USA. Celem modeli będzie ceny ubezpieczenia w zależności od wartości parametrów opisujących daną osobę.

In [ ]:
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Proszę:
<li>Utworzyć folder "AI_datasets" w lokalizacji "Mój Dysk"</li>
<li>Dodać plik 'insurance.csv' z moodle.</li>

In [ ]:
%cd /content/gdrive/My Drive/AI_datasets

In [ ]:
!ls

In [ ]:
insurance = pd.read_csv("insurance_kz.csv")

In [ ]:
insurance.shape

In [ ]:
insurance.head()

In [ ]:
insurance.describe()

In [ ]:
sns.pairplot(insurance)

In [ ]:
corr = insurance.corr(method = 'spearman')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},annot=True)

In [ ]:
import scipy.stats

In [ ]:
sns.displot(insurance, x="charges", kind="kde", bw_adjust=.1)

In [ ]:
upper_bound = (insurance['charges'].quantile(0.75)+1.5*scipy.stats.iqr(insurance['charges']))
lower_bound = (insurance['charges'].quantile(0.25)-1.5*scipy.stats.iqr(insurance['charges']))
print(upper_bound,lower_bound)

In [ ]:
sns.displot(insurance.loc[(insurance['charges']<upper_bound)&
                                       (insurance['charges']>lower_bound)],
             x="charges",
             kind="kde",
             bw_adjust=.2)

In [ ]:
insurance_full = insurance.copy()
insurance.drop(insurance.loc[(insurance['charges']>28000)|
                                       (insurance['charges']<lower_bound)].index,axis = 0,inplace=True)
insurance.reset_index(drop=True,inplace=True)

In [ ]:
insurance.shape

In [ ]:
insurance['sex'].value_counts()

In [ ]:
insurance['region'].value_counts()

In [ ]:
insurance['smoker'].value_counts()

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.barplot(x="region", y="charges", data=insurance)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.barplot(x="sex", y="charges", data=insurance)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")

In [ ]:
import plotly.figure_factory as ff

hist_data =[]
for region in insurance['region'].unique():
  hist_data.append(insurance.loc[insurance['region']==region,'charges'].tolist())

group_labels = insurance['region'].unique()


# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False,show_rug=False)

# Add title
fig.update_layout(title_text='density')
fig.show()

In [ ]:
hist_data =[]
for sex in insurance['sex'].unique():
  hist_data.append(insurance.loc[insurance['sex']==sex,'charges'].tolist())

group_labels = insurance['sex'].unique()


# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False,show_rug=False)

# Add title
fig.update_layout(title_text='density')
fig.show()

In [ ]:
hist_data =[]
for smoker in insurance['smoker'].unique():
  hist_data.append(insurance.loc[insurance['smoker']==smoker,'charges'].tolist())

group_labels = insurance['smoker'].unique()


# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False,show_rug=False)

# Add title
fig.update_layout(title_text='density')
fig.show()

## Normalizacja i one hot encoding

In [ ]:
insurance = pd.get_dummies(insurance, columns=['sex','region','smoker'], prefix = ['sex','region','smoker'],drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
insurance.columns

In [ ]:
insurance.head()

In [ ]:
scale_factors = {'min':insurance.charges.min() , 'max' : insurance.charges.max()}

In [ ]:
scale_factors

In [ ]:
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(insurance.loc[:,['age', 'bmi', 'children', 'charges']]) 
insurance = insurance.copy()
insurance.loc[:,['age', 'bmi', 'children', 'charges']] = scaled_values

In [ ]:
insurance.head()

In [ ]:
insurance.describe()

## Predykcje opłat mieszczących się w normie

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = insurance.drop('charges',axis = 1).copy()
y = insurance.loc[:,'charges']

In [ ]:
train_X, test_X, train_y, test_y= train_test_split(X,y, test_size=0.2, random_state=8)

In [ ]:
X.shape

In [ ]:
train_X.shape

In [ ]:
test_X.shape

In [ ]:
def create_nn_regression_model(optimizer,
                               layers,
                               input_dim = X.shape[1]):
  model = Sequential()
  model.add(Dense(layers[0][0],
          input_dim=input_dim,
          activation=layers[0][1]))
  for neurons, activation_function in layers[1:]:
    model.add(Dense(neurons,
                    activation=activation_function))

  model.add(Dense(1, activation=linear,
                  kernel_initializer='normal'))

  # define avaluation and optimization criteria
  model.compile(loss=mean_squared_error,
                optimizer=optimizer,
                metrics=['mse'])
  return model

In [ ]:
model = create_nn_regression_model(SGD(),
                                   [(1000,elu),(800,elu),(600,elu),(400,elu),(200,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=100,
                    batch_size=100,
                    validation_split=0.2,
                    verbose=1)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
model = create_nn_regression_model(SGD(),
                                   [(100,elu),(80,elu),(60,elu),(40,elu),(20,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=100,
                    batch_size=100,
                    validation_split=0.2,
                    verbose=0)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
model = create_nn_regression_model(SGD(),
                                   [(100,elu),(60,elu),(20,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=200,
                    batch_size=100,
                    validation_split=0.2,
                    verbose=0)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
model = create_nn_regression_model(Adam(),
                                   [(100,elu),(60,elu),(20,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=200,
                    batch_size=100,
                    validation_split=0.2,
                    verbose=0)

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
model = create_nn_regression_model(Adadelta(learning_rate = 0.1),
                                   [(100,elu),(60,elu),(20,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=100,
                    batch_size=100,
                    validation_split=0.2,
                    verbose=0)

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()


In [ ]:
model = create_nn_regression_model(RMSprop(),
                                   [(100,elu),(60,elu),(20,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=200,
                    batch_size=100,
                    validation_split=0.2,
                    verbose=0)

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

W procesie uczenia możliwe jest zdefiniowanie dodatkowych własności. Jedną z nich jest <b>EarlyStopping</b>, który monitoruje zmiany podanego parametru w kolejnych iteracjach. Jeżeli są one zbyt niskie (model przestaje się uczyć), proces uczenia jest zatrzymany, a jako końcowe wagi w neuronach zwracane są te, dla któych wartość wskazanego parametru jest najbardziej korzystna (np. najwyższa precyzja, najniższy błąd).

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            min_delta=0.0001, 
                                            patience=300, 
                                            verbose=0, 
                                            mode='min', 
                                            restore_best_weights=True)

In [ ]:
model = create_nn_regression_model(Adam(),
                                   [(100,elu),(60,elu),(20,elu)]
                                   )
history = model.fit(train_X,
                    train_y,
                    epochs=1000,
                    batch_size=100,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=0)

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()


## Nie taki black box

In [ ]:
predictions = model.predict(test_X).reshape(test_y.shape)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=predictions, y=test_y,
                    mode='markers', name = 'train'))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_y, y=predictions-test_y,
                    mode='markers', name = 'train'))
fig.show()


In [ ]:
real_y_pred = predictions.reshape(test_y.shape)*(scale_factors['max']-scale_factors['min'])+scale_factors['min']
real_y_test = test_y*(scale_factors['max']-scale_factors['min'])+scale_factors['min']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=real_y_test, y=real_y_pred-real_y_test,
                    mode='markers', name = 'train'))
fig.show()

Model uczy się na danych znormalizowanych, więc predykcja nie może być interpretowalna jako wielkość opłat. Możemy jednak zmienić wagi ostatniej warstwy modelu, aby predykcje odpowiadały rzeczywistym wartośćiom.

In [ ]:
modified_model = tf.keras.models.clone_model(model)
modified_model_weights = model.get_weights()

In [ ]:
modified_model_weights

In [ ]:
modified_model_weights[-2] = modified_model_weights[-2] * (scale_factors['max'] - scale_factors['min'])
modified_model_weights[-1] = modified_model_weights[-1] * (scale_factors['max'] - scale_factors['min'])+scale_factors['min']

In [ ]:
modified_model_weights

In [ ]:
modified_model.set_weights(modified_model_weights)

In [ ]:
real_y_pred = modified_model.predict(test_X).reshape(test_y.shape)#+scale_factors['min']
real_y_test = test_y*(scale_factors['max']-scale_factors['min'])+scale_factors['min']

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=real_y_test, y=real_y_pred-real_y_test,
                    mode='markers', name = 'train'))
fig.show()

### Shap values

A może coś nowocześniejszego? Metoda SHAP została przedstawiona w 2017 roku (<a href = https://arxiv.org/pdf/1705.07874.pdf>artykuł</a>) i aktualnie jest jedną z częściej używanych metod objaśniania modeli ML. Jej użyteczność wynika z:<ul>
<li>elastyczności - można jej używać w lasach losowych, XGB, modelach analizy sentymentu czy właśnie głębokich sieciach neurnonowych</li>
<li>możliwości objaśnienia łącznego wpływu zmiennych na model jak i wyliczenia takich wartości dla pojedynczych obserwacji</li>
<li>mocnym oparciu w teorii - w tle używamy kombinacji przybliżenia modelami liniowymi oraz teorii gier</li></ul>

In [ ]:
!pip install shap
import shap

shap.initjs()

In [ ]:
train_X.head()

In [ ]:
test_X.shape

In [ ]:
explainer = shap.KernelExplainer(model, test_X)
data_to_explain = shap.sample(test_X,100)
shap_values = explainer.shap_values(data_to_explain)

In [ ]:
shap.summary_plot(shap_values,data_to_explain, plot_type = 'bar')

In [ ]:
shap.summary_plot(shap_values[0],data_to_explain)

In [ ]:
shap.initjs()
shap.plots.force(explainer.expected_value[0],shap_values[0],data_to_explain)

In [ ]:
shap.initjs()
single_obs = data_to_explain.iloc[0]
single_shap_values = explainer.shap_values(single_obs)
shap.plots.force(explainer.expected_value[0],single_shap_values[0],single_obs)

In [ ]:
explain_single_obs = shap.Explanation(values = single_shap_values[0],
                                      base_values = explainer.expected_value[0],
                                      data = single_obs,
                                      feature_names = data_to_explain.columns)
shap.plots.waterfall(explain_single_obs)

## DALEX

In [ ]:
import dalex as dx

In [ ]:
exp = dx.Explainer(model, X, y)

In [ ]:
exp.model_performance().plot()

In [ ]:
exp.model_parts().plot()

In [ ]:
exp.model_profile().plot()

In [ ]:
exp.model_diagnostics().plot()

#### Ćwiczenie

Ustaw EarlyStopping, żeby był cierpliwy przez 100 iteracji, max iteracje na 400 i min_delta = 0.0001. Sprawdź:<ul>
<li>czy model jest bardziej przetrenowany?</li>
<li>jak liczba wpłynęła na rokład błędu predykcji?</li>
<li>czy wpływ zmiennych na predykcję się zmienił?</li></ul>

### Zadanie 6 
Jak model poradzi sobie ze zbiorem danych, w którym uwzględnione zostały wszystkie osoby, bez względu na ich opłaty za ubezpieczenie? Wytrenuj model przy użyciu zbioru <i>insurance_full</i> po odpowiednim przygotowaniu danych oraz przeanalizuj wyniki predykcji. Jak bardzo wpływowe są wartości skrajne?

# Klasyfikacja

## Normalizacja i one hot encoding

In [ ]:
insurance_full.columns

In [ ]:
insurance_full_dummy = pd.get_dummies(insurance_full, columns=['sex', 'smoker', 'region'], prefix = ['sex', 'smoker', 'region'],drop_first=True)

In [ ]:
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(insurance_full_dummy.loc[:,['age', 'bmi', 'children', 'charges']]) 
insurance_full_minmax = insurance_full_dummy.copy()
insurance_full_minmax.loc[:,['age', 'bmi', 'children', 'charges']] = scaled_values

Zamieńmy zmienną 'charges' na zero jedynkową:
0 - cena "w normie", czyli poniżej 1m
1 - wysokie ceny, powyżej 1m

In [ ]:
X = insurance.drop('charges',axis = 1).copy()
y = insurance.loc[:,'charges']

In [ ]:
y.hist()

In [ ]:
y = (y>0.1)*1

In [ ]:
sum(y)

In [ ]:
len(y)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y= train_test_split(X,y, test_size=0.2, random_state=8)

## Model klasyfikacyjny

In [ ]:
def create_nn_classification_model(optimizer,
                               layers,
                               input_dim = X.shape[1]):
  model = Sequential()
  model.add(Dense(layers[0][0],
          input_dim=input_dim,
          activation=layers[0][1]))
  for neurons, activation_function in layers[1:]:
    model.add(Dense(neurons,
                    activation=activation_function))

  model.add(Dense(1, activation=sigmoid,
                  kernel_initializer='normal'))

  # define avaluation and optimization criteria
  model.compile(loss=binary_crossentropy,
                optimizer=optimizer,
                metrics=['acc','AUC'])
  return model

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_auc', 
                                            min_delta=0.01, 
                                            patience=50, 
                                            verbose=0, 
                                            mode='max', 
                                            restore_best_weights=True)

In [ ]:
model = create_nn_classification_model(Adam(),
                                       [(100,elu),(80,elu),(60,elu),(40,elu),(20,elu)])

In [ ]:
history = model.fit(train_X,
                    train_y,
                    epochs=50,
                    batch_size=100,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=1)

In [ ]:
# Wydobycie wartości funkcji kosztu z pliku json
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
# Wydobycie wartości funkcji kosztu z pliku json
loss = history.history['acc']
val_loss = history.history['val_acc']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
# Wydobycie wartości funkcji kosztu z pliku json
loss = history.history['auc']
val_loss = history.history['val_auc']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
y.hist()

In [ ]:
predictions = model.predict(test_X)

In [ ]:
np.mean(predictions)

In [ ]:
np.median(predictions)

In [ ]:
pd.Series(predictions.reshape(len(predictions))).hist()

In [ ]:
y_predicted = predictions>0.35

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# Moduł sklearn.metrics pozwala na automatyczne obliczenie bardziej skomplikowanych metryk.
print('Confusion Matrix')
print(confusion_matrix(test_y, y_predicted))
print('Classification Report')
print(classification_report(test_y, y_predicted))
print('Accuracy = '+ str(accuracy_score(test_y, y_predicted)))

In [ ]:
lr_fpr, lr_tpr, _ = roc_curve(test_y, predictions)
# plot the roc curve for the model
plt.plot(lr_fpr, lr_tpr, marker='.', label='Neural Network')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[*range(len(predictions))], y=predictions.reshape(len(predictions)),
                         marker=dict(
                color=test_y.to_list()),
                    mode='markers', name = 'train'))
fig.show()

W zadaniach klasyfikacyjnych często zdarza się, że udział obiektów z różnych klas nie jest równy. W takim przypadku model może bardzo dobrze odwzorowywać etykietę większościową, jednak predykcje dla etykiet mniejszościowych będą obciążone. Używając <b>class_weight</b> wagi poszczególnych etykiet w podanym zbiorze danych są automatycznie wyliczane . 

In [ ]:
from sklearn.utils import compute_class_weight

In [ ]:
class_weights = dict(zip(np.unique(train_y),
                         compute_class_weight(
                                    class_weight = "balanced",
                                    classes = np.unique(train_y),
                                    y = train_y                                                    
                                    )
                         ))

In [ ]:
class_weights

In [ ]:
model = create_nn_classification_model(Adam(),
                                       [(100,elu),(80,elu),(60,elu),(40,elu),(20,elu)])

In [ ]:
history = model.fit(train_X,
                    train_y,
                    epochs=50,
                    batch_size=100,
                    class_weight = class_weights,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    verbose=1)

#predictions = model.predict(X)

In [ ]:
# Wydobycie wartości funkcji kosztu z pliku json
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
# Wydobycie wartości funkcji kosztu z pliku json
loss = history.history['auc']
val_loss = history.history['val_auc']
epochs = range(len(loss))

fig = go.Figure()
fig.add_trace(go.Scatter(x=[*epochs], y=loss,
                    mode='lines', name = 'train'))
fig.add_trace(go.Scatter(x=[*epochs], y=val_loss,
                    mode='lines', name = 'val'))
fig.show()

In [ ]:
predictions = model.predict(test_X)

In [ ]:
pd.Series(predictions.reshape(len(predictions))).hist()

In [ ]:
y_predicted = predictions>0.5

# Moduł sklearn.metrics pozwala na automatyczne obliczenie bardziej skomplikowanych metryk.
print('Confusion Matrix')
print(confusion_matrix(test_y, y_predicted))
print('Classification Report')
print(classification_report(test_y, y_predicted))
print('Accuracy = '+ str(accuracy_score(test_y, y_predicted)))

In [ ]:
lr_fpr, lr_tpr, _ = roc_curve(test_y, predictions)
# plot the roc curve for the model
plt.plot(lr_fpr, lr_tpr, marker='.', label='Neural Network')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[*range(len(predictions))], y=predictions.reshape(len(predictions)),
                         marker=dict(
                color=test_y.to_list()),
                    mode='markers', name = 'train'))
fig.show()

In [ ]:
explainer = shap.KernelExplainer(model, shap.kmeans(test_X,100))
data_to_explain = shap.sample(test_X,100)
shap_values = explainer.shap_values(data_to_explain)

In [ ]:
shap.summary_plot(shap_values,data_to_explain, plot_type = 'bar')

In [ ]:
shap.summary_plot(shap_values[0],data_to_explain)

# \* Szukanie najlepszych parametrów sieci z użyciem tensorboard

In [ ]:
from tensorboard.plugins.hparams import api as hp

In [ ]:
%load_ext tensorboard

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 64]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([2,3]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','RMSprop']))
HP_ACTIVATION_FUNCTION = hp.HParam('activation_function', hp.Discrete(['relu','tanh']))

METRIC_AUC = 'AUC'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_NUM_LAYERS, HP_OPTIMIZER,HP_ACTIVATION_FUNCTION],
    metrics=[hp.Metric(METRIC_AUC, display_name='AUC')],
  )

In [ ]:
!ls

In [ ]:
def train_test_model(hparams,
                     train_X = train_X,
                     train_y = train_y,
                     test_X = test_X,
                     test_y = test_y,
                     class_weights = class_weights,
                     early_stop = early_stop,
                     input_dim = X.shape[1]):
  model = Sequential()
  model.add(Dense(hparams[HP_NUM_UNITS],
          input_dim=input_dim,
          activation=elu))
  for i in range(hparams[HP_NUM_LAYERS]):
    model.add(Dense(hparams[HP_NUM_UNITS],
                    activation=hparams[HP_ACTIVATION_FUNCTION]))

  model.add(Dense(1, activation='sigmoid',
                  kernel_initializer='normal'))
  

  # define avaluation and optimization criteria
  model.compile(loss=binary_crossentropy,
                optimizer=hparams[HP_OPTIMIZER],
                metrics=['AUC'])
  model.fit(train_X,
            train_y,
            epochs=10,
            class_weight = class_weights,
            callbacks=[early_stop],
            verbose=0) 
  _, auc = model.evaluate(test_X, test_y)
  return auc

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    auc = train_test_model(hparams)
    tf.summary.scalar(METRIC_AUC,auc , step=1)

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                            min_delta=0.01, 
                                            patience=15, 
                                            verbose=0, 
                                            mode='min', 
                                            restore_best_weights=True)

In [ ]:
rm -rf ./logs/

In [ ]:
session_num = 0


for num_layers in HP_NUM_LAYERS.domain.values:
  for num_units in HP_NUM_UNITS.domain.values:    
    for activation_function in HP_ACTIVATION_FUNCTION.domain.values:
      for optimizer in HP_OPTIMIZER.domain.values:
        hparams = {
            HP_NUM_UNITS: num_units,
            HP_NUM_LAYERS: num_layers,
            HP_OPTIMIZER: optimizer,
            HP_ACTIVATION_FUNCTION: activation_function
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1

In [ ]:
%tensorboard --logdir logs/hparam_tuning

# Poćwicz w domu
<ul>
<li>Zadanie 5* z wykorzystaniem tensorboard</li>
<li>Dlaczego pijemy wino? Wytrenuj model predykcyjny przy użyciu DNN, który będzie przewidywał wydatki na wino. Przeanalizuj wpływ poszczególnych zmiennych przy użyciu SHAP values.</li>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
marketing_campaign = pd.read_csv("marketing_campaign.csv",sep='\t')

In [ ]:
marketing_campaign.head()

In [ ]:
zmienne_niezależne = ['Year_Birth','Education','Marital_Status','Income','Kidhome','Teenhome']
zmienna_zależna = ['MntWines']

In [ ]:
marketing_campaign = marketing_campaign[zmienna_zależna+zmienne_niezależne]

In [ ]:
marketing_campaign.head()

In [ ]:
marketing_campaign = marketing_campaign.loc[~np.isin(marketing_campaign.Marital_Status,['Alone','Absurd', 'YOLO'])].copy()